In [1]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np

In [2]:
import pickle
def save_obj(obj, name):
       with open(name + '.pkl', 'wb') as f:
           pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
       with open(name + '.pkl', 'rb') as f:
           return pickle.load(f)

In [3]:
clases = 5

In [ ]:
# Cargamos la planilla con las páginas a escrapear
webs = pd.ExcelFile("Webs.xls")

In [ ]:
print(webs.sheet_names)

In [ ]:
df = webs.parse("Categorias")
clases_df = set(df["CÓDIGO"].value_counts().index)

In [ ]:
df

In [ ]:
clases_df

## Paso 1 - Scraping de textos crudos

In [ ]:
textos = pd.DataFrame(columns=['Texto', 'Clase'])
for i in df.index: 
    print("i", i)
    url = df["URL"][i]
    req = get(url)
    statusCode = req.status_code
    html = BeautifulSoup(req.text, "html.parser")
    if statusCode == 200:
        paragraphs = html.select("p")
        cont = 0
        for para in paragraphs:
            cont += 1 
            parrafo = ""
            print("parrafo",parrafo) 
            print("i",i) 
            print("df['CÓDIGO'][i]",df["CÓDIGO"][i]) 
            print('df["NOMBRE ARCHIVO"][i]',df["NOMBRE ARCHIVO"][i])
            print("cont",cont) 
            guardarComo = str(df["CÓDIGO"][i]) + "/"  + df["NOMBRE ARCHIVO"][i] + str(cont)
            print("guardarComo",guardarComo)  
            parrafo = open(guardarComo,"w",encoding = "utf-8") 
            parrafo.write(para.text)
            textos = textos.append({'Texto': para.text, 'Clase':df["CÓDIGO"][i]}, ignore_index=True)
            parrafo.close() 
            clear_output( wait = True )
        print(f"DataFrame {i} de {len(df.index)}")
        print()
        print(para)       

In [ ]:
textos

In [ ]:
textos.groupby(by=["Clase"]).count()

## Paso 2 - Preprocesamiento de textos

In [ ]:
textos = pd.read_csv("H01 - HistoriaTextosCrudos.csv")
textos = textos.iloc[:,1:]
textos.head(10)

In [ ]:
stop        = stopwords.words('english')
tokenizer   = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
stemmer     = PorterStemmer()

textosP      = pd.DataFrame(columns = [ 'Texto' , 'Clase'])

for i in textos.index:
    print()    
    texto_tokens = tokenizer.tokenize(textos["Texto"][i])
    texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation ]
    texto_stem   = [ stemmer.stem(word) for word in texto_clean ]
    textosP   = textosP.append({'Texto' : texto_stem, 'Clase' : textos["Clase"][i] }, ignore_index = True)
    clear_output( wait = True )
    print(f"Procesando {i} de {len(textos.index)}")
    print(textos["Texto"][i])
    print(texto_tokens)
    print() 
    print (texto_clean)
    print() 
    print (texto_stem)
textosP

In [ ]:
textos = textosP
textos

## Extracción de las palabras confusas

**Vamos a cargar la lista de palabras confusas y las vamos a extraer de los textos**

In [ ]:
confusas = load_obj("Confusas")

In [ ]:
confusas

In [ ]:
len(confusas)

In [ ]:
cont = 0
for i in textos.index:
    cont += 1
    if cont > 50000:
        break
    # print()
    # print()
    # print(textos["Texto"][i])
    # print()
    # print("largo texto:",len(textos["Texto"][i]))
    # print()
    # print(textos["Texto"][i])
    # print()
    lista = []
    for word in textos["Texto"][i]:
        # print(word)
        if word in confusas:
            print("La palabra",word,"está en el listado de las palabras confusas")
        else:
            lista.append(word) 
    print()
    print("largo texto:",len(textos["Texto"][i]))
    print("largo lista sin confusas:",len(lista))
    print()

In [ ]:
for i in textos.index:
    lista = []
    for word in textos["Texto"][i]:
        if word not in confusas:
            lista.append(word) 
    clear_output( wait = True )
    print(f"Procesando {i} de {len(textos.index)}") 
    print()
    print("largo texto:",len(textos["Texto"][i]))
    print("largo lista sin confusas:",len(lista))
    print()
    textos["Texto"][i] = lista
textos

In [ ]:
textos20 = pd.DataFrame(columns = ['Texto' , 'Clase'])

cont = 0
for i in textos.index:
    for t in range(len(textos["Texto"][i]) // 15):
        j = 20 * t
        palabras   = textos["Texto"][i][ j : 15 * (t+1) ] 
        textos20   = textos20.append({'Texto' : palabras, 'Clase' : textos["Clase"][i] }, ignore_index = True)
        clear_output( wait = True )
        print(f"Creando Textos {i} de {len(textos.index)}")
textos20

In [ ]:
textos20['Texto' ][0]

In [ ]:
len(textos20['Texto' ][0])

In [ ]:
textos = textos20

In [ ]:
textos.to_csv("Historia.csv")

In [ ]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

In [ ]:
textos.groupby(by=["Clase"]).count()

## Vocabulario y frecuencias

In [ ]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

In [ ]:
textos.groupby(by=["Clase"]).count()

In [ ]:
vocabulario = set() 
for m in textos.index:
    lista = textos.loc[m].Texto.split(",")
    cont2 = 0
    for word in lista:
        cont2 += 1
        if cont2 == len(lista):
            word = word[:-1] 
        word = word[2:-1]   
        if word[-1:] == "'":
            word = word[:-1]
        vocabulario.add(word)
        clear_output( wait = True )
        print(f"Creando Vocabulario {m} de {len(textos.index)}")

In [ ]:
len(vocabulario)

In [ ]:
"hunter-gather" in vocabulario

In [ ]:
save_obj(vocabulario, "Vocabulario")

In [ ]:
for word in vocabulario:
    print(word)    

In [ ]:
vocabulario = load_obj("Vocabulario")

In [ ]:
frecuencias = defaultdict(int)
cont = 0 
for i in textos.index:
    cont += 1
    print()
    print("Contador",cont)
    if cont > 10:
        break
    lista = textos.loc[i].Texto.split(",")
    cont2 = 0
    for word in lista:
        cont2 += 1
        if cont2 == len(lista):
            word = word[:-1] 
        word = word[2:-1]   
        if word[-1:] == "'":
            word = word[:-1]
        print(word)
        # word = "pirulito"
        assert (word in vocabulario),"La palabra no está en el vocabulario!"
        frecuencias[(word, textos.loc[i].Clase)] += 1
        clear_output( wait = True )
        print(f"Creando Frecuencias {i} de {len(textos)}")

In [ ]:
frecuencias = defaultdict(int)
for i in textos.index:
    lista = textos.loc[i].Texto.split(",")
    cont2 = 0
    for word in lista:
        cont2 += 1
        if cont2 == len(lista):
            word = word[:-1] 
        word = word[2:-1]   
        if word[-1:] == "'":
            word = word[:-1]
        print(word)
        # word = "pirulito"
        assert (word in vocabulario),"La palabra no está en el vocabulario!"
        frecuencias[(word, textos.loc[i].Clase)] += 1
        clear_output( wait = True )
        print(f"Creando Frecuencias {i} de {len(textos)}")

In [ ]:
save_obj(frecuencias, "Frecuencias")

In [ ]:
frecuencias = load_obj("Frecuencias")

In [ ]:
len(frecuencias)

In [ ]:
frecuencias

In [ ]:
def limpiarWord(word):
    word = word[:-1]
    return word

In [ ]:
# create frequency dictionary
freqs = {}
cont = 0
for i in textos.index:  
    cont += 1
    if cont > 100000:
        break
    print()    
    print(i)    
    words = textos.loc[i,'Texto'].split(",")
    clase = textos.loc[i,'Clase']
    cont2 = 0
    for word in words:
        cont2 += 1
        if cont2 == len(words):
            word = word[:-1] 
        word = word[2:-1]   

      
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        
        print(word)
        # word = "pirulito"
        assert (word in vocabulario),"La palabra " + word + "no está en el vocabulario!"
        pair = (word, clase)
        freqs[pair] = freqs.get(pair, clase) + 1
# check data type
print(f'type(freqs) = {type(freqs)}')

# check length of the dictionary
print(f'len(freqs) = {len(freqs)}')  

In [ ]:
print(textos["Texto"][60])

In [ ]:
print(textosCrudos["Texto"][30])

In [ ]:
# calculate V, the number of unique words in the vocabulary
vocab = set([pair[0] for pair in freqs.keys()])
V = len(vocab)
print(V)

In [ ]:
save_obj(freqs, "Frecuencias")

In [ ]:
frecuencias = load_obj("Frecuencias")

In [ ]:
len(frecuencias)

In [ ]:
def extract_features(words, freqs, vocabulario):
    '''
    Input: 
        words: a list of words for one input
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,6)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = words
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 6)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    # loop through each word in the list of words
    cont2 = 0
    for word in word_l:
        cont2 += 1
        if cont2 == len(words):
            word = word[:-1] 
        word = word[2:-1]   
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        print(word)
        assert (word in vocabulario),"La palabra " + word + "no está en el vocabulario!"

        # increment the word count for the positive label 1
        for i in range(1,6):
            x[0,i] += freqs.get((word, i),0)        
    assert(x.shape == (1, 6))
    return x

In [ ]:
textosFreq = textos.copy()
for j in range(1, clases + 1):
    textosFreq["Clase" + str(j)] = None
textosFreq    

In [ ]:
for i in textosFreq .index:
    words = textos.loc[:,"Texto"][i].split(",")    
    clases = extract_features(words, freqs, vocabulario)
    nclases = len(clases[0]) 
    for j in range(1, nclases):
        textosFreq["Clase" + str(j)][i] = clases[0,j]
textosFreq        

In [ ]:
textosFreq.sample(10)

In [13]:
textosFreq.to_csv("H03 - HistoriaFreq.csv")

NameError: name 'textosFreq' is not defined

In [15]:
textos = pd.read_csv("H03 - HistoriaFreq.csv")
textos = textos.iloc[:,1:]
textos

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
0,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",1,458.0,204.0,135.0,119.0,47.0
1,"['alexand', 'greek', 'ὁ', 'μέγας', 'ho', 'méga...",1,802.0,350.0,276.0,66.0,97.0
2,"['philip', 'ii', 'throne', 'age', 'rule']",1,270.0,341.0,457.0,96.0,244.0
3,"['alexand', 'tutor', 'aristotl', 'age', ""phili...",1,1096.0,478.0,557.0,314.0,434.0
4,"['pan-hellen', 'greek', 'conquest', '4', '334'...",1,825.0,459.0,190.0,158.0,245.0
...,...,...,...,...,...,...,...
11477,"['herd', 'now-extinct', 'pleistocen', 'megafau...",5,11.0,7.0,10.0,6.0,167.0
11478,"['hunter-gather', 'america', 'base', 'plain', ...",5,176.0,248.0,298.0,503.0,322.0
11479,"['lifestyl', 'group', 'style', 'stone', 'tool'...",5,154.0,123.0,119.0,97.0,666.0
11480,"['archaic', 'period', 'america', 'environ', 'f...",5,216.0,218.0,221.0,237.0,768.0


In [17]:
textosDrop=pd.DataFrame(columns=['Texto','Clase','Clase1','Clase2','Clase3','Clase4','Clase5'])
cont = 0
for i in textos.index:
    cont += 1
    if cont > 1000000:
        break
    print ("LARGO" ,len(textos["Texto"][i].split())  )    
    if len(textos["Texto"][i].split()) >= 15:
        textosDrop = textosDrop.append({
'Texto':textos["Texto"][i],'Clase':textos['Clase'][i],'Clase1':textos['Clase1'][i]
        ,'Clase2':textos['Clase2'][i],'Clase3':textos['Clase3'][i]
        ,'Clase4':textos['Clase4'][i],'Clase5':textos['Clase5'][i]}, ignore_index=True)
textosDrop        

LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5

LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15


LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LA

LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 

LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 1

...

LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARG

LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 15
LARGO 10

LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 15
LARGO 15
L

LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 5
LARGO 15
LARGO 10
LARGO 15
LARGO 10
LARGO 15
LARGO 10


,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
0,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",1,458.0,204.0,135.0,119.0,47.0
1,"['alexand', 'tutor', 'aristotl', 'age', ""phili...",1,1096.0,478.0,557.0,314.0,434.0
2,"['alexand', 'endeavour', 'sea', 'invad', 'indi...",1,792.0,297.0,305.0,317.0,265.0
3,"[""alexander'"", 'cultur', 'diffus', 'syncret', ...",1,592.0,358.0,205.0,97.0,518.0
4,"['alexand', 'born', 'pella', 'capit', 'kingdom...",1,735.0,311.0,270.0,290.0,188.0
...,...,...,...,...,...,...,...
5404,"['agriculturalist', 'hunt', 'gather', 'farm', ...",5,124.0,122.0,139.0,170.0,491.0
5405,"['nevertheless', 'hunter-gather', 'contact', '...",5,271.0,443.0,235.0,147.0,470.0
5406,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",5,442.0,431.0,534.0,498.0,2010.0
5407,"['hunter-gather', 'america', 'base', 'plain', ...",5,176.0,248.0,298.0,503.0,322.0


In [6]:
textosDrop

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
0,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",1,458.0,204.0,135.0,119.0,47.0
1,"['alexand', 'greek', 'ὁ', 'μέγας', 'ho', 'méga...",1,802.0,350.0,276.0,66.0,97.0
2,"['philip', 'ii', 'throne', 'age', 'rule']",1,270.0,341.0,457.0,96.0,244.0
3,"['alexand', 'tutor', 'aristotl', 'age', ""phili...",1,1096.0,478.0,557.0,314.0,434.0
4,"['pan-hellen', 'greek', 'conquest', '4', '334'...",1,825.0,459.0,190.0,158.0,245.0
...,...,...,...,...,...,...,...
11477,"['herd', 'now-extinct', 'pleistocen', 'megafau...",5,11.0,7.0,10.0,6.0,167.0
11478,"['hunter-gather', 'america', 'base', 'plain', ...",5,176.0,248.0,298.0,503.0,322.0
11479,"['lifestyl', 'group', 'style', 'stone', 'tool'...",5,154.0,123.0,119.0,97.0,666.0
11480,"['archaic', 'period', 'america', 'environ', 'f...",5,216.0,218.0,221.0,237.0,768.0


In [7]:
textos = textosDrop

In [8]:
textos

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
0,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",1,458.0,204.0,135.0,119.0,47.0
1,"['alexand', 'greek', 'ὁ', 'μέγας', 'ho', 'méga...",1,802.0,350.0,276.0,66.0,97.0
2,"['philip', 'ii', 'throne', 'age', 'rule']",1,270.0,341.0,457.0,96.0,244.0
3,"['alexand', 'tutor', 'aristotl', 'age', ""phili...",1,1096.0,478.0,557.0,314.0,434.0
4,"['pan-hellen', 'greek', 'conquest', '4', '334'...",1,825.0,459.0,190.0,158.0,245.0
...,...,...,...,...,...,...,...
11477,"['herd', 'now-extinct', 'pleistocen', 'megafau...",5,11.0,7.0,10.0,6.0,167.0
11478,"['hunter-gather', 'america', 'base', 'plain', ...",5,176.0,248.0,298.0,503.0,322.0
11479,"['lifestyl', 'group', 'style', 'stone', 'tool'...",5,154.0,123.0,119.0,97.0,666.0
11480,"['archaic', 'period', 'america', 'environ', 'f...",5,216.0,218.0,221.0,237.0,768.0


In [9]:
textos.to_csv("H03 - HistoriaFreq.csv")

In [10]:
textos = pd.read_csv("H03 - HistoriaFreq.csv")
textos = textos.iloc[:,1:]
textos

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
0,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",1,458.0,204.0,135.0,119.0,47.0
1,"['alexand', 'greek', 'ὁ', 'μέγας', 'ho', 'méga...",1,802.0,350.0,276.0,66.0,97.0
2,"['philip', 'ii', 'throne', 'age', 'rule']",1,270.0,341.0,457.0,96.0,244.0
3,"['alexand', 'tutor', 'aristotl', 'age', ""phili...",1,1096.0,478.0,557.0,314.0,434.0
4,"['pan-hellen', 'greek', 'conquest', '4', '334'...",1,825.0,459.0,190.0,158.0,245.0
...,...,...,...,...,...,...,...
11477,"['herd', 'now-extinct', 'pleistocen', 'megafau...",5,11.0,7.0,10.0,6.0,167.0
11478,"['hunter-gather', 'america', 'base', 'plain', ...",5,176.0,248.0,298.0,503.0,322.0
11479,"['lifestyl', 'group', 'style', 'stone', 'tool'...",5,154.0,123.0,119.0,97.0,666.0
11480,"['archaic', 'period', 'america', 'environ', 'f...",5,216.0,218.0,221.0,237.0,768.0


In [11]:
textos.sample(10)

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
7983,"['idea', 'underpin', 'modern', 'meritocraci', ...",4,260.0,269.0,251.0,502.0,432.0
2553,"['william', 'ockham', 'manner', 'earlier', 'fr...",2,40.0,258.0,145.0,63.0,76.0
8370,"['widespread', 'chemic', 'featur', 'conflict',...",4,203.0,196.0,266.0,656.0,91.0
6564,"['1685', 'maria', 'antonia', 'spanish', 'thron...",3,111.0,449.0,784.0,272.0,88.0
11470,"['hunter-gather', 'conscious', 'manipul', 'lan...",5,124.0,68.0,60.0,49.0,462.0
450,"['mention', 'alban', 'hous', 'rome', '7th', 'c...",1,742.0,445.0,234.0,71.0,226.0
7985,"['inquisit', 'holi', 'roman', 'empir', 'power'...",4,767.0,738.0,418.0,343.0,64.0
5559,"['foreign', 'devil', 'gloriou', 'king', 'hail']",3,169.0,232.0,230.0,47.0,14.0
8962,"[""kennedy'"", 'foreign', 'polici', 'american', ...",4,86.0,108.0,150.0,622.0,117.0
8500,"['1931', 'loan', 'repaid', 'britain', 'owe']",4,33.0,30.0,38.0,100.0,20.0


In [12]:
textos

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
0,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",1,458.0,204.0,135.0,119.0,47.0
1,"['alexand', 'greek', 'ὁ', 'μέγας', 'ho', 'méga...",1,802.0,350.0,276.0,66.0,97.0
2,"['philip', 'ii', 'throne', 'age', 'rule']",1,270.0,341.0,457.0,96.0,244.0
3,"['alexand', 'tutor', 'aristotl', 'age', ""phili...",1,1096.0,478.0,557.0,314.0,434.0
4,"['pan-hellen', 'greek', 'conquest', '4', '334'...",1,825.0,459.0,190.0,158.0,245.0
...,...,...,...,...,...,...,...
11477,"['herd', 'now-extinct', 'pleistocen', 'megafau...",5,11.0,7.0,10.0,6.0,167.0
11478,"['hunter-gather', 'america', 'base', 'plain', ...",5,176.0,248.0,298.0,503.0,322.0
11479,"['lifestyl', 'group', 'style', 'stone', 'tool'...",5,154.0,123.0,119.0,97.0,666.0
11480,"['archaic', 'period', 'america', 'environ', 'f...",5,216.0,218.0,221.0,237.0,768.0


In [18]:
textos = textosDrop

In [19]:
textos.to_csv("H03 - HistoriaFreq.csv")

In [20]:
textos = pd.read_csv("H03 - HistoriaFreq.csv")
textos = textos.iloc[:,1:]
textos

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
0,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",1,458.0,204.0,135.0,119.0,47.0
1,"['alexand', 'tutor', 'aristotl', 'age', ""phili...",1,1096.0,478.0,557.0,314.0,434.0
2,"['alexand', 'endeavour', 'sea', 'invad', 'indi...",1,792.0,297.0,305.0,317.0,265.0
3,"[""alexander'"", 'cultur', 'diffus', 'syncret', ...",1,592.0,358.0,205.0,97.0,518.0
4,"['alexand', 'born', 'pella', 'capit', 'kingdom...",1,735.0,311.0,270.0,290.0,188.0
...,...,...,...,...,...,...,...
5404,"['agriculturalist', 'hunt', 'gather', 'farm', ...",5,124.0,122.0,139.0,170.0,491.0
5405,"['nevertheless', 'hunter-gather', 'contact', '...",5,271.0,443.0,235.0,147.0,470.0
5406,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",5,442.0,431.0,534.0,498.0,2010.0
5407,"['hunter-gather', 'america', 'base', 'plain', ...",5,176.0,248.0,298.0,503.0,322.0


In [21]:
textos.sample(10)

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
2357,"['dutch', 'british', 'fed', 'led', 'movement',...",3,117.0,511.0,810.0,582.0,157.0
3646,"['keen', 'observ', ""bonaparte'"", 'power', 'mad...",4,225.0,234.0,355.0,499.0,103.0
5379,"['endur', 'hypothesi', 'long-dist', 'hunt', 'p...",5,260.0,233.0,270.0,317.0,1004.0
5091,"['complex', 'mode', 'tool', 'acheulean', 'indu...",5,204.0,284.0,285.0,275.0,840.0
2268,"['christian', 'embassi', 'sent', 'karakorum', ...",3,133.0,325.0,211.0,149.0,90.0
2202,"['ottoman', 'empir', 'conquer', 'territori', '...",2,528.0,695.0,640.0,360.0,126.0
4499,"['petroglyph', 'appear', 'neolith', 'petroglyp...",5,191.0,139.0,129.0,84.0,758.0
2739,"['elizabeth', 'establish', 'english', 'church'...",3,160.0,431.0,565.0,352.0,115.0
5133,"['harappan', 'period', 'characteris', 'wall', ...",5,421.0,381.0,284.0,166.0,835.0
1741,"['franc', 'civil', 'war', 'deadli', 'epidem', ...",2,449.0,656.0,712.0,1039.0,156.0


In [22]:
textos.sample(10)

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
3715,"['potassium', 'tartrat', 'treatment', 'prevent...",4,42.0,28.0,54.0,372.0,82.0
5043,"['exampl', 'associ', 'el', 'jobo', 'venezuela'...",5,109.0,115.0,108.0,85.0,452.0
5292,"['chesowanja', 'archaeologist', 'found', 'red'...",5,213.0,135.0,100.0,68.0,649.0
3170,"['diseas', 'killer', 'indian', 'practic', 'sla...",3,340.0,350.0,572.0,369.0,221.0
3386,"['610', 'deputi', 'sat', 'estat', 'theori', 'v...",4,139.0,182.0,157.0,212.0,292.0
617,"['marcu', 'aureliu', 'philosoph', 'good', 'emp...",1,537.0,424.0,418.0,623.0,71.0
4087,"['china', 'nationalist', 'communist', 'forc', ...",4,542.0,443.0,621.0,1337.0,146.0
3873,"['front', 'mi', 'pari', 'krupp', 'railway', 'g...",4,105.0,137.0,148.0,351.0,163.0
4563,"['terramar', 'indo-european', 'civil', 'area',...",5,303.0,368.0,279.0,274.0,392.0
1828,"['clotair', 'ii', '613', 'reunit', 'frankish',...",2,208.0,329.0,196.0,254.0,104.0
